# Demo for automated computation of binary phase diagram

In [1]:
from pyiron_workflow.graph import gui, base

In [2]:
pf = gui.PyironFlow(["water"]) 
pf.gui

In [ ]:
j = pr.create.job.Lammps('water_equilibration', delete_existing_job = True)

solvated_electrode.add_tag(selective_dynamics=[True, True, True])
solvated_electrode.selective_dynamics[solvated_electrode.select_index("Al")] = [False, False, False]

epsilon = 0.102
sigma = 3.188
water_potential = pandas.DataFrame({
    'Name': ['H2O_tip3p'],
    'Filename': [[]],
    'Model': ["TIP3P"],
    'Species': [['H','O','Al']],
    'Config': [[
    '# @potential_species H_O  ### species in potential\n',
     '# W.L. Jorgensen',
     'The Journal of Chemical Physics 79',
     '926 (1983); https://doi.org/10.1063/1.445869 \n',
     '#\n',
     '\n',
     'units      real\n',
     'dimension  3\n',
     'atom_style full\n',
     '\n',
     '# create groups ###\n',
     'group O type 2\n',
     'group H type 1\n',
     'group Al type 3\n',
     '\n',
     '## set charges - beside manually ###\n',
     'set group O charge -0.830\n',
     'set group H charge 0.415\n',
     'set group Al charge 0.2\n',           
     '\n',
     '### TIP3P Potential Parameters ###\n',
     'pair_style lj/cut/coul/long 10.0\n',
     'pair_coeff * * 0.000 0.000 \n',
     'pair_coeff 2 2 0.102 3.188 \n',
     'pair_coeff 2 3 {:.4} {:.4} \n'.format(epsilon, sigma),      
     'bond_style  harmonic\n',
     'bond_coeff  1 450 0.9572\n',
     'angle_style harmonic\n',
     'angle_coeff 1 55 104.52\n',
     'kspace_style pppm 1.0e-5   # final npt relaxation\n',
     '\n']]})

j.structure = solvated_electrode
j.potential = water_potential
j.calc_md(temperature=300)   

j.run(run_mode='queue', delete_existing_job=True)

In [ ]:
xx

In [6]:
import pyiron_database.instance_database as idb
from pyiron_nodes.math import Sin

sin = Sin(3)

idb.store_node_outputs

<function pyiron_database.instance_database.node.store_node_outputs(node: pyiron_workflow.simple_workflow.Node) -> str>

In [3]:
pf = gui.PyironFlow(["phonopy", "show_code", "phonopy_macro", "phonopy_free_energy", "elastic", "elastic_macro", "elastic_macro"]) 
pf.gui

Adding macro node phonopy


In [3]:
pf = gui.PyironFlow(["phonopy", "show_code", "phonopy_macro", "phonopy_free_energy", "elastic", "elastic_macro", "elastic_macro2"]) 
pf.gui

Adding macro node phonopy
Adding macro node ComputeElasticConstantsMacro
Adding macro node CubicBulkCell


Expanding node ComputeElasticConstantsMacro


In [3]:
from pyiron_nodes.atomistic.property.elastic import ComputeElasticConstants



In [11]:
from pyiron_workflow import Workflow, Node
import pyiron_nodes as pn

wf = Workflow("test")
wf.node = Node()
wf.sin = pn.math.Sin(x=3)

wf.run(), wf.node

AttributeError: 'Node' object has no attribute 'function'

In [28]:
from pyiron_workflow import as_macro_node, as_function_node, Node
from pyiron_nodes.atomistic.structure.build import Bulk
from pyiron_nodes.atomistic.engine.ase import M3GNet
from pyiron_nodes.atomistic.calculator.ase import StaticEnergy, Static
from pyiron_nodes.atomistic.property.elastic import InputElasticTensor, SymmetryAnalysis, GenerateStructures, AnalyseStructures

@as_function_node
def ComputeElasticConstants(
    structure,
    engine,
    # calculator: Node = None,    
    input_elastic_tensor: InputElasticTensor = None,
):
    """
    Get the elastic constants of a structure using an ASE calculator.
    """
    from pyiron_workflow import Workflow
    from pyiron_nodes.controls import iterate, IterToDataFrame, Print
    from pyiron_nodes.atomistic.calculator.ase import StaticEnergy, Static
    from pyiron_nodes.atomistic.property.phonons import GetFreeEnergy

    wf = Workflow("elastic_constants")
    if input_elastic_tensor is None:
        input_elastic_tensor = InputElasticTensor()
    # wf.print = Print(f"calculator: {calculator}")
    # wf.calculator = StaticEnergy(structure=structure, engine=engine)
    wf.calculator = GetFreeEnergy(structure=structure, engine=engine)
    print(f"Using calculator: {wf.calculator}")
    # print(f"Input calculator: {calculator}")
    wf.symmetry = SymmetryAnalysis(structure=structure, parameters=input_elastic_tensor)
    wf.structures = GenerateStructures(
        structure=structure, analysis=wf.symmetry, parameters=input_elastic_tensor
    )
    wf.energies = iterate(
        node=wf.calculator,
        values=wf.structures.outputs.structures,
        input_label="structure",
    )

    wf.elastic_constants = AnalyseStructures(
        energies=wf.energies,
        job_names=wf.structures.outputs.job_names,
        analysis=wf.symmetry,
        parameters=input_elastic_tensor,
    )
    elastic_constants = wf.energies.pull()
    return elastic_constants

bulk = Bulk("Al", cubic=True).run()
engine = M3GNet().run()
calculator = StaticEnergy(bulk, engine=engine) #.run()
ComputeElasticConstants(bulk, engine, calculator).run()    

Using calculator: <pyiron_workflow.simple_workflow.Node object at 0x199e15c70>


KeyError: 'InputElasticTensor'

In [17]:
from pyiron_workflow import Workflow
from pyiron_nodes.controls import iterate, IterToDataFrame, Print
from pyiron_nodes.atomistic.calculator.ase import StaticEnergy, Static
from pyiron_nodes.atomistic.property.phonons import GetFreeEnergy

from pyiron_workflow import as_macro_node, as_function_node, Node
from pyiron_nodes.atomistic.structure.build import Bulk
from pyiron_nodes.atomistic.engine.ase import M3GNet
from pyiron_nodes.atomistic.calculator.ase import StaticEnergy, Static
from pyiron_nodes.atomistic.property.elastic import InputElasticTensor, SymmetryAnalysis, GenerateStructures, AnalyseStructures

structure = Bulk("Al", cubic=True).run()
engine = M3GNet().run()
calculator = StaticEnergy(structure, engine=engine)
calculator = GetFreeEnergy(structure=structure, engine=engine)
parameters = InputElasticTensor(num_of_point=6)

@as_function_node
def elastic_constants(structure, calculator: Node, input_elastic_tensor:InputElasticTensor=None):
    wf = Workflow("elastic_constants")
    if input_elastic_tensor is None:
        input_elastic_tensor = InputElasticTensor().run()
    wf.calculator = calculator 
    
    wf.symmetry = SymmetryAnalysis(structure=structure, parameters=input_elastic_tensor)
    wf.structures = GenerateStructures(
        structure=structure, analysis=wf.symmetry, parameters=input_elastic_tensor
    )
    wf.energies = iterate(
        node=wf.calculator,
        values=wf.structures.outputs.structures,
        input_label="structure",
    )
    
    wf.elastic_constants = AnalyseStructures(
        energies=wf.energies,
        job_names=wf.structures.outputs.job_names,
        analysis=wf.symmetry,
        parameters=input_elastic_tensor,
    )
    
    elastic_constants = wf.elastic_constants.pull()
    return elastic_constants

print(calculator)
node = elastic_constants(structure, calculator, parameters)
node.run()

/Users/joerg/miniforge3/envs/intel12/lib/python3.12/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/joerg/miniforge3/envs/intel12/lib/python3.12/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/joerg/miniforge3/envs/intel12/lib/python3.12/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather th

Error getting hash for node: GetFreeEnergy maximum recursion depth exceeded
copy node:  GetFreeEnergy None
Calculating free energy at 300 K


/Users/joerg/miniforge3/envs/intel12/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!
DE

Free energy calculated for temperature: 300 K:  {'temperatures': array([300.]), 'free_energy': array([-12.62283435]), 'entropy': array([145.58394598]), 'heat_capacity': array([96.10179911])}
Free energy: -12.622834348976632 eV
Error getting hash for node: calculator maximum recursion depth exceeded
copy node:  calculator None
Calculating free energy at 300 K
Free energy calculated for temperature: 300 K:  {'temperatures': array([300.]), 'free_energy': array([-12.62283435]), 'entropy': array([145.58394598]), 'heat_capacity': array([96.10179911])}
Free energy: -12.622834348976632 eV
Calculating free energy at 300 K
Free energy calculated for temperature: 300 K:  {'temperatures': array([300.]), 'free_energy': array([-11.57095222]), 'entropy': array([142.34089824]), 'heat_capacity': array([95.84778349])}
Free energy: -11.570952222607874 eV
Calculating free energy at 300 K
Free energy calculated for temperature: 300 K:  {'temperatures': array([300.]), 'free_energy': array([-12.09915514]), '

OutputElasticAnalysis(strain_energy=[[(-0.005, -11.570952222607874), (-0.0025, -12.099155141040244), (0.0, -12.622834348976632), (0.0024999999999999996, -13.150752300052268), (0.005, -13.691761552666597)], [(-0.005, -11.922914483538875), (-0.0025, -12.273889760954768), (0.0, -12.622834348976632), (0.0024999999999999996, -12.974226131818785), (0.005, -13.328334913980633)], [(-0.005, -14.578606296405589), (-0.0025, -14.586610285541344), (0.0, -12.622834348976632), (0.0024999999999999996, -14.30955963976287), (0.005, -17.136706752394797)]], C=array([[    68.15210251,   -308.34768129,   -308.34768129,
             0.        ,      0.        ,      0.        ],
       [  -308.34768129,     68.15210251,   -308.34768129,
             0.        ,      0.        ,      0.        ],
       [  -308.34768129,   -308.34768129,     68.15210251,
             0.        ,      0.        ,      0.        ],
       [     0.        ,      0.        ,      0.        ,
        -42672.20130306,      0.      

In [3]:
from phonopy import Phonopy
from structuretoolkit.common import atoms_to_phonopy, phonopy_to_atoms

from pyiron_nodes.atomistic.structure.build import CubicBulkCell

Al = CubicBulkCell('Al', 3).run()
phonopy = Phonopy(unitcell=atoms_to_phonopy(Al))
phonopy.generate_displacements(distance=0.01, is_plusminus='auto', is_diagonal=True, is_trigonal=False, number_of_snapshots=None, random_seed=None, temperature=None, cutoff_frequency=None, max_distance=None)


DEBUG:pyiron_log:Not supported parameter used!
DEBUG:pyiron_log:Not supported parameter used!


In [2]:
from pyiron_nodes.atomistic.property.phonons import ThermalProperties

ThermalProperties().dataclass()

ThermalProperties(temperatures=array([], dtype=float64), free_energy=array([], dtype=float64), entropy=array([], dtype=float64), heat_capacity=array([], dtype=float64))

In [17]:
from pyiron_workflow import as_out_dataclass_node
import numpy as np

@as_out_dataclass_node
class ThermalProperties:
    from dataclasses import field

    temperatures: list | np.ndarray = field(
        default_factory=lambda: np.array([])
    )

ThermalProperties().dataclass(temperatures=[1])

ThermalProperties(temperatures=[1])

In [4]:
from pyiron_nodes.atomistic.property.phonons import PhonopyParameters
from dataclasses import asdict

pp = PhonopyParameters().run()
asdict(pp)

{'distance': 0.01,
 'is_plusminus': 'auto',
 'is_diagonal': True,
 'is_trigonal': False,
 'number_of_snapshots': None,
 'random_seed': None,
 'temperature': None,
 'cutoff_frequency': None,
 'max_distance': None}

In [3]:
pf = gui.PyironFlow(['assyst',  'linearfit2', 'landau2']) 
pf.gui

In [6]:
layout = gui.GUILayout()
layout.flow_widget_height = 800

# working: 'Workflow_4', 'experiment', 'assyst_linear_fit3', 'neighbors1', 'energy', 'murn4', 'db'

In [7]:
pf = gui.PyironFlow(['Workflow_4', 'assyst_linear_fit3', 'macro_fit', 'bspline_test', 'bspline_test3', 'iter_test', 'descriptor'], gui_layout=layout) 
pf.gui

Adding macro node CubicBulkCell
Adding macro node FitDiffPotential


In [8]:
xx

NameError: name 'xx' is not defined

In [ ]:
import pyiron_nodes as pn

Al = pn.atomistic.structure.build.Bulk('Al')
Al.inputs.name.value = "Cu"
Al.inputs["name"].value ="Fe"
Al.inputs

In [ ]:
import ast
from typing import Tuple, List, Dict, Any
from pyiron_workflow import Node
from pyiron_workflow.simple_workflow import Data, Port, PORT_LABEL, PORT_DEFAULT, PORT_TYPE, PORT_VALUE


### BEGIN: Helper function from previous answer with stricter checks
class ReturnAnalysisError(Exception):
    pass

def analyze_function_code(func_str):
    tree = ast.parse(func_str)
    # Find the first function definition
    func_node = next(node for node in tree.body if isinstance(node, ast.FunctionDef))
    # Arguments
    args_info = []
    all_args = func_node.args.args
    defaults = [None] * (len(all_args) - len(func_node.args.defaults)) + func_node.args.defaults
    for arg, default in zip(all_args, defaults):
        arg_name = arg.arg
        arg_type = ast.unparse(arg.annotation) if arg.annotation else None
        if default is not None:
            try:
                default_value = ast.literal_eval(default)
            except Exception:
                default_value = ast.unparse(default)
        else:
            default_value = None
        args_info.append({
            'name': arg_name,
            'type': arg_type,
            'default': default_value
        })
    # Return type annotation
    return_type = ast.unparse(func_node.returns) if func_node.returns else None

    # Extract return variable names from 'return' (with safety checks)
    class ReturnVisitor(ast.NodeVisitor):
        def __init__(self):
            self.return_vars = []
        def visit_Return(self, node):
            if node.value is None:
                self.return_vars.append(None)
            elif isinstance(node.value, ast.Name):
                self.return_vars.append(node.value.id)
            elif isinstance(node.value, ast.Tuple):
                names = []
                for elt in node.value.elts:
                    if isinstance(elt, ast.Name):
                        names.append(elt.id)
                    else:
                        raise ReturnAnalysisError(
                            f"Invalid return variable: {ast.unparse(elt)}. Must return variable names, not expressions."
                        )
                self.return_vars.append(tuple(names))
            else:
                raise ReturnAnalysisError(
                    f"Invalid return value: {ast.unparse(node.value)}. Must return variable names, not expressions."
                )
    visitor = ReturnVisitor()
    visitor.visit(func_node)
    if len(visitor.return_vars) > 1:
        raise ReturnAnalysisError("Function contains multiple 'return' statements, which is not supported.")
    return_vars = visitor.return_vars[0] if visitor.return_vars else None
    return {
        'function_name': func_node.name,
        'arguments': args_info,
        'return_type': return_type,
        'returned_variables': return_vars
    }
### END: Helper function

### The requested main function:
def function_string_to_node(func_str):
    # Analyze the function code to extract port specifications and return variable names
    info = analyze_function_code(func_str)
    arg_info = info['arguments']
    return_type = info['return_type']
    return_vars = info['returned_variables']
    # Compose code string and function object
    local_vars = {}
    exec_globals = {
        '__builtins__': __builtins__,
        'Tuple': Tuple,
        'List': List,
        'Dict': Dict,
        'Any': Any,
    }
    exec(func_str, exec_globals, local_vars)
    fn = [v for k,v in local_vars.items() if callable(v)][0]  # Gets the defined function
    
    # Prepare inputs for Node
    inputs = Data({
        PORT_LABEL: [arg['name'] for arg in arg_info],
        PORT_TYPE: [arg['type'] for arg in arg_info],
        PORT_DEFAULT: [arg['default'] for arg in arg_info],
        PORT_VALUE: [None] * len(arg_info),
        "ready": [False] * len(arg_info)
    }, attribute=Port)

    # Prepare outputs for Node
    if return_vars is None:
        output_labels = []
        output_types = []
    elif isinstance(return_vars, str):
        output_labels = [return_vars]
        output_types = [return_type]
    else:  # tuple of names
        output_labels = list(return_vars)
        # split return_type if it's a tuple type (e.g. Tuple[int,int])
        # Otherwise just assign the same return_type to all outputs for lack of better info
        if return_type and return_type.startswith('Tuple['):
            inside = return_type[6:-1]
            out_types = [s.strip() for s in inside.split(',')]
            if len(out_types) == len(output_labels):
                output_types = out_types
            else:
                output_types = [return_type] * len(output_labels)
        else:
            output_types = [return_type] * len(output_labels)
    outputs = Data({
        PORT_LABEL: output_labels,
        PORT_TYPE: output_types,
        PORT_VALUE: [None]*len(output_labels),
        "ready": [False] * len(output_labels)
    }, attribute=Port)
    
    node = Node(
        func=fn,
        inputs=inputs,
        outputs=outputs,
        label=info['function_name'],
        output_labels=output_labels if output_labels else None,
        node_type="function_node"
    )
    return node




In [ ]:
# Test with an example

func_str = '''
def AddMultiply(x: float, y: int = 2) -> Tuple[float, float]:
    z = x + y
    w = x * y
    return z, w
'''
n = function_string_to_node(func_str)
n(x=3, y=2)
n.outputs.z.value

In [ ]:
from typing import Any, Dict, List, Optional

PORT_LABEL = 'label'
PORT_TYPE = 'type'
PORT_DEFAULT = 'default'
PORT_VALUE = 'value'

class Port:
    """Holds the attributes of a single port."""
    def __init__(
        self,
        label: str,
        type_: Optional[Any] = None,
        default: Optional[Any] = None,
        value: Optional[Any] = None
    ) -> None:
        self.label = label
        self.type = type_
        self.default = default
        self.value = value

    def __repr__(self) -> str:
        return (f"Port(label={self.label!r}, type={self.type!r}, "
                f"default={self.default!r}, value={self.value!r})")


class Data:
    """Holds a set of Port objects, mapped by their label (as attributes)."""
    def __init__(self, port_dict: Dict[str, List[Any]]) -> None:
        # Determine number of ports:
        primary_key = PORT_LABEL
        labels = port_dict.get(primary_key, [])
        # get the keys in port_dict that will be attributes of Port:
        keys = list(port_dict.keys())
        for i, label in enumerate(labels):
            attrs = {}
            for key in keys:
                value_list = port_dict.get(key, [])
                attrs[key] = value_list[i] if i < len(value_list) else None
            # Create Port instance with unpacked dict and add it as attribute
            self.__setattr__(label, Port(
                label=attrs.get(PORT_LABEL),
                type_=attrs.get(PORT_TYPE),
                default=attrs.get(PORT_DEFAULT),
                value=attrs.get(PORT_VALUE)
            ))

    def __repr__(self) -> str:
        ports = [attr for attr in self.__dict__ if isinstance(getattr(self, attr), Port)]
        return f"Data({', '.join(ports)})"


# --- Example usage ---
if __name__ == "__main__":
    input = Data({
        PORT_LABEL: ['label_1', 'label_2'],
        PORT_TYPE: ['int', 'float'],
        PORT_DEFAULT: [0, 1.0],
        PORT_VALUE: [None, None],
    })
    port_1 = input.label_1
    print(port_1)              # Port(label='label_1', type='int', default=0, value=None)
    print(port_1.label)        # 'label_1'
    print(port_1.type)         # 'int'
    print(port_1.default)      # 0
    print(port_1.value)        # None

    port_2 = input.label_2
    print(port_2)              # Port(label='label_2', type='float', default=1.0, value=None)

In [ ]:
from pyiron_workflow import as_function_node, Workflow
from pyiron_workflow.simple_workflow import _return_as_function_node


@as_function_node
def CodeToNode(code):
    node = function_string_to_node(code)
    return code


wf = Workflow('CodeEditor')
wf.add_multiply = CodeToNode(code=func_str)   

 

In [ ]:
def sin(x: float):
    import numpy as np
    sin = np.sin(x)
    return sin

_return_as_function_node(sin,'Sin', ['sin'], 'function_node')

In [ ]:
sin_func()

In [ ]:
import scipy

from scipy.interpolate import BSpline

BSpline.design_matrix??

In [ ]:
from pyiron_nodes.atomistic.structure.calc import FitDiffPotential2

FitDiffPotential2()

In [ ]:
from pyiron_workflow import Workflow
from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
    ReadPickledDatasetAsDataframe,
)
from pyiron_nodes.math import Linspace, Divide, DotProduct
from pyiron_nodes.atomistic.structure.calc import LinearInterpolationDescriptor
from pyiron_nodes.dataframe import (
    MergeDataFrames,
    GetRowsFromDataFrame,
    GetColumnFromDataFrame,
    ApplyFunctionToSeriesNew,
)
from pyiron_nodes.math import Subtract, PseudoInverse, Sum, DotProduct

file_path_0: str = "ASSYST/Al_LDA.pckl.gz"
file_path_1: str = "ASSYST/Al_PBE.pckl.gz"
r_min: float = 2.5
r_max: float = 7
num_points: int = 51
max_row_index: int = -1
store = False


wf = Workflow("assyst_linear_fit3")

wf.ReadData = ReadPickledDatasetAsDataframe(
    file_path=file_path_0,
    compression="gzip",
)

wf.ReadRefData = ReadPickledDatasetAsDataframe(
    file_path=file_path_1,
    compression="gzip",
)

wf.Linspace = Linspace(x_min=r_min, x_max=r_max, num_points=num_points)

wf.MergeDataFrames = MergeDataFrames(
    df1=wf.ReadData,
    df2=wf.ReadRefData,
    on="name",
    how="inner",
)

wf.LinearInterpolationDescriptor = LinearInterpolationDescriptor(r_bins=wf.Linspace)

wf.GetRowsFromDataFrame = GetRowsFromDataFrame(
    df=wf.MergeDataFrames, max_index=max_row_index
)

wf.GetStructures = GetColumnFromDataFrame(
    df=wf.GetRowsFromDataFrame, column_name="ase_atoms_x"
)

wf.NumberOfAtoms = GetColumnFromDataFrame(
    df=wf.GetRowsFromDataFrame, column_name="NUMBER_OF_ATOMS_x"
)

wf.GetEnergy = GetColumnFromDataFrame(
    df=wf.GetRowsFromDataFrame, column_name="energy_corrected_y"
)

wf.GetRefEnergy = GetColumnFromDataFrame(
    df=wf.GetRowsFromDataFrame, column_name="energy_corrected_x"
)

wf.DesignMatrix = ApplyFunctionToSeriesNew(
    series=wf.GetStructures,
    function=wf.LinearInterpolationDescriptor,
    store=store,
)

wf.DiffEnergy = Subtract(x=wf.GetEnergy, y=wf.GetRefEnergy)

wf.PseudoInverse = PseudoInverse(matrix=wf.DesignMatrix)

wf.Sum = Sum(x=wf.DesignMatrix, axis=0)

wf.Coeff = DotProduct(a=wf.PseudoInverse, b=wf.DiffEnergy, store=store)
wf.DiffEnergyPerAtom = Divide(wf.DiffEnergy, wf.NumberOfAtoms)
wf.FitEnergyDiff = DotProduct(a=wf.DesignMatrix, b=wf.Coeff)
wf.FitEnergyDiffPerAtom = Divide(wf.FitEnergyDiff, wf.NumberOfAtoms)

wf.MergeDataFrames.pull()

In [ ]:
@as_macro_node(["coefficients", "design_matrix", "r_bins", "diff_energy_per_atom", "fit_diff_energy_per_atom", "number_of_atoms"])
def FitDiffPotential(
    file_path_0: str = "ASSYST/Al_LDA.pckl.gz",
    file_path_1: str = "ASSYST/Al_PBE.pckl.gz",
    r_min: float = 2.5,
    r_max: float = 7,
    num_points: int = 51,
    max_row_index: int = -1,
    store: bool = True,
):

    from pyiron_workflow import Workflow
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        ReadPickledDatasetAsDataframe,
    )
    from pyiron_nodes.math import Linspace, Divide, DotProduct
    from pyiron_nodes.atomistic.structure.calc import LinearInterpolationDescriptor
    from pyiron_nodes.dataframe import (
        MergeDataFrames,
        GetRowsFromDataFrame,
        GetColumnFromDataFrame,
        ApplyFunctionToSeriesNew,
    )
    from pyiron_nodes.math import Subtract, PseudoInverse, Sum, DotProduct

    wf = Workflow("assyst_linear_fit3")

    wf.ReadData = ReadPickledDatasetAsDataframe(
        file_path=file_path_0,
        compression="gzip",
    )

    wf.ReadRefData = ReadPickledDatasetAsDataframe(
        file_path=file_path_1,
        compression="gzip",
    )

    wf.Linspace = Linspace(x_min=r_min, x_max=r_max, num_points=num_points)

    wf.MergeDataFrames = MergeDataFrames(
        df1=wf.ReadData,
        df2=wf.ReadRefData,
        on="name",
        how="inner",
    )

    wf.LinearInterpolationDescriptor = LinearInterpolationDescriptor(r_bins=wf.Linspace)

    wf.GetRowsFromDataFrame = GetRowsFromDataFrame(
        df=wf.MergeDataFrames, max_index=max_row_index
    )

    wf.GetStructures = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name="ase_atoms_x"
    )

    wf.NumberOfAtoms = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name="NUMBER_OF_ATOMS_x"
    )

    wf.GetEnergy = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name="energy_corrected_y"
    )

    wf.GetRefEnergy = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name="energy_corrected_x"
    )

    wf.DesignMatrix = ApplyFunctionToSeriesNew(
        series=wf.GetStructures,
        function=wf.LinearInterpolationDescriptor,
        store=store,
    )

    wf.DiffEnergy = Subtract(x=wf.GetEnergy, y=wf.GetRefEnergy)

    wf.PseudoInverse = PseudoInverse(matrix=wf.DesignMatrix)

    wf.Sum = Sum(x=wf.DesignMatrix, axis=0)

    wf.Coeff = DotProduct(a=wf.PseudoInverse, b=wf.DiffEnergy, store=store)
    wf.DiffEnergyPerAtom = Divide(wf.DiffEnergy, wf.NumberOfAtoms)
    wf.FitEnergyDiff = DotProduct(a=wf.DesignMatrix, b=wf.Coeff)
    wf.FitEnergyDiffPerAtom = Divide(wf.FitEnergyDiff, wf.NumberOfAtoms)

    return wf.Coeff, wf.DesignMatrix, wf.Linspace, wf.DiffEnergyPerAtom, wf.FitEnergyDiffPerAtom, wf.NumberOfAtoms


In [ ]:
xx

In [ ]:
pf = gui.PyironFlow(['assyst_data', 'neighbors1', 'assyst_linear_fit3', 'experiment', 'Workflow_4'], gui_layout=layout) 
pf.gui

In [ ]:
xx

In [ ]:
from pyiron_workflow import Workflow
from pyiron_workflow.graph.base import (Graph, graph_to_code, collapse_node, Nodes, Node, GraphEdge, 
get_code_from_graph, GraphNode, get_updated_graph, collapse_node, add_node, _remove_virtual_edges, _remove_edges_to_hidden_nodes, copy_graph)
from pyiron_workflow.graph.graph_json import _save_graph, _load_graph
from pyiron_workflow.graph.gui import GuiGraph, _mark_node_as_collapsed

wf = Workflow('test')

from pyiron_nodes.atomistic.structure.build import CubicBulkCell, Bulk
from pyiron_nodes.atomistic.structure.transform import Repeat

structure = CubicBulkCell("Al")
# structure = Bulk("Al")

graph = Graph(label="test")
graph += structure

graph_c = collapse_node(graph, "CubicBulkCell")

state = graph_c.__getstate__()  #["nodes"]["CubicBulkCell"]
del state["nodes"]["CubicBulkCell"]["graph"]
state["nodes"]["CubicBulkCell"]["node"] = graph.nodes["CubicBulkCell"]["node"].__getstate__()
state["nodes"]["CubicBulkCell"]["node_type"] = "node"
state["edges"]["values"] = []
state

nodes = Nodes().__setstate__(state["nodes"])
nodes
#new_graph = Graph().__setstate__(state)



#_save_graph(graph, overwrite=True)
#_load_graph("test.json")
# print (get_code_from_graph(graph.nodes["CubicBulkCell"].graph, sort_graph=True, use_node_default=False))
# print(graph_to_code(graph))


In [ ]:
from copy import copy

structure = CubicBulkCell("Al")
repeat = Repeat()

graph = Graph(label="test")
graph = add_node(graph, structure, label="CubicBulk")
graph = add_node(graph, repeat, label="repeat1")
graph += GraphEdge(source="CubicBulk", target="repeat1", sourceHandle="structure", targetHandle="structure")


def compact_graph(graph: Graph):
    graph = copy_graph(graph)
    for k, node in graph.nodes.items():
        # find macro nodes in the top level and collapse them
        if (node.graph is not None) and (node.parent_id is None) and (node.import_path is not None):
            print("collapse: ", k)
            new_node = GraphNode(node=node.node, 
                                 id=node.id,
                                 label=node.label, 
                                 expanded=False, 
                                 import_path=node.import_path, 
                                 node_type=node.node_type)
            graph.nodes[k] = new_node
            graph = collapse_node(graph, k)
            
    graph = get_updated_graph(graph)
    return graph 


cg = compact_graph(graph)
state = cg.__getstate__()
# state["edges"]


In [ ]:
def uncompact_graph_from_state(state):
    graph = Graph(label=state["label"])
    for k, node_state in state["nodes"].items():
        if isinstance(node_state, dict):
            # print(k, type(node_state))
            graph_node = GraphNode().__setstate__(node_state)
            if (graph_node.node is None) and (graph_node.import_path is not None):
                node = Node().__setstate__(node_state["node"])
                graph = add_node(graph, node, label=node.label)
                graph = _mark_node_as_collapsed(graph, node.label)
            else:
                graph +=graph_node

    for edge_state in state["edges"]["values"]:
        print(edge_state)
        graph += GraphEdge(**edge_state)

    return graph

new_graph = uncompact_graph_from_state(state)

In [ ]:
pf = gui.PyironFlow([new_graph]) 
pf.gui

In [ ]:
u_graph.edges

In [ ]:
GuiGraph(new_graph)

In [ ]:
Nodes().__setstate__(state["nodes"])

In [ ]:
GuiGraph(cg)

In [ ]:
graph = Graph(label="test")

state["nodes"]["CubicBulkCell"]["expanded"] = False
node = GraphNode().__setstate__(state["nodes"]["CubicBulkCell"]) #["node"])

node = Node().__setstate__(state["nodes"]["CubicBulkCell"]["node"])

graph = add_node(graph, node, label=state["nodes"]["CubicBulkCell"]["label"])
node.expanded = False

graph = collapse_node(graph, "CubicBulkCell")

graph = get_updated_graph(graph)
node = graph.nodes["CubicBulkCell"]
node.graph = None
#node.node_type = "node"
print(graph_to_code(graph))

# _save_graph(graph, overwrite=True)
# _load_graph("test.json")

In [ ]:
graph.__getstate__()

In [ ]:
GuiGraph(get_updated_graph(graph))

In [ ]:
from pyiron_workflow.graph.gui import GuiGraph

GuiGraph(graph.nodes["CubicBulkCell"].graph)

In [ ]:
state["nodes"]["CubicBulkCell"]["node"]
Node().__setstate__(state["nodes"]["CubicBulkCell"]["node"])

In [ ]:
from pyiron_workflow import Workflow
from pyiron_workflow.graph.base import Graph, graph_to_code, collapse_node, Nodes, Node
from pyiron_workflow.graph.graph_json import _save_graph, _load_graph

wf = Workflow('test')

from pyiron_nodes.atomistic.structure.build import CubicBulkCell, Bulk

#structure = CubicBulkCell("Al")
structure = Bulk("Al")

graph = Graph(label="test")
graph += structure

state = graph.__getstate__() 
#del state["nodes"]["CubicBulkCell"]["graph"]
#state["nodes"]["CubicBulkCell"]["node"] = graph.nodes["CubicBulkCell"]["node"].__getstate__()
#state["edges"]["values"] = []
#state

nodes = Nodes().__setstate__(state["nodes"])
nodes

In [ ]:
from pyiron_workflow.graph.base import get_graph_from_wf
from pyiron_workflow.graph.gui import GuiGraph
from pyiron_workflow.graph.graph_json import _save_graph, _load_graph

graph = get_graph_from_wf(wf, wf_outputs=[wf.structure], out_labels=["structure"])

# _save_graph(graph, overwrite=True)
# _load_graph("test.json")
# GuiGraph(graph)

In [ ]:
from pyiron_workflow import as_macro_node

@as_macro_node("figure")
def assyst_linear_fit3(
    ReadPickledDatasetAsDataframe__file_path: str = "ASSYST/Al_LDA.pckl.gz",
    ReadPickledDatasetAsDataframe__compression: str = "gzip",
    ReadPickledDatasetAsDataframe_1__file_path: str = "ASSYST/Al_PBE.pckl.gz",
    ReadPickledDatasetAsDataframe_1__compression: str = "gzip",
    Linspace__x_min: float = 2.5,
    Linspace__x_max: float = 7,
    Linspace__num_points: int = 51,
    MergeDataFrames__on: str = "name",
    MergeDataFrames__how: str = "inner",
    GetRowsFromDataFrame__max_index: int = -1,
    GetColumnFromDataFrame_2__column_name: str = "ase_atoms_x",
    GetColumnFromDataFrame_3__column_name: str = "energy_corrected_y",
    GetColumnFromDataFrame_1__column_name: str = "energy_corrected_x",
    ApplyFunctionToSeries_2__store: bool = True,
    Sum__axis: int = 0,
    DotProduct__store: bool = True,
):

    from pyiron_workflow import Workflow

    wf = Workflow("assyst_linear_fit3")

    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        ReadPickledDatasetAsDataframe,
    )

    wf.ReadPickledDatasetAsDataframe = ReadPickledDatasetAsDataframe(
        file_path=ReadPickledDatasetAsDataframe__file_path,
        compression=ReadPickledDatasetAsDataframe__compression,
    )
    from pyiron_nodes.atomistic.ml_potentials.fitting.linearfit import (
        ReadPickledDatasetAsDataframe,
    )

    wf.ReadPickledDatasetAsDataframe_1 = ReadPickledDatasetAsDataframe(
        file_path=ReadPickledDatasetAsDataframe_1__file_path,
        compression=ReadPickledDatasetAsDataframe_1__compression,
    )
    from pyiron_nodes.math import Linspace

    wf.Linspace = Linspace(
        x_min=Linspace__x_min, x_max=Linspace__x_max, num_points=Linspace__num_points
    )
    from pyiron_nodes.dataframe import MergeDataFrames

    wf.MergeDataFrames = MergeDataFrames(
        df1=wf.ReadPickledDatasetAsDataframe,
        df2=wf.ReadPickledDatasetAsDataframe_1,
        on=MergeDataFrames__on,
        how=MergeDataFrames__how,
    )
    from pyiron_nodes.atomistic.structure.calc import LinearInterpolationDescriptor

    
    wf.LinearInterpolationDescriptor = LinearInterpolationDescriptor(r_bins=wf.Linspace)
    from pyiron_nodes.dataframe import GetRowsFromDataFrame

    wf.GetRowsFromDataFrame = GetRowsFromDataFrame(
        df=wf.MergeDataFrames, max_index=GetRowsFromDataFrame__max_index
    )
    from pyiron_nodes.dataframe import GetColumnFromDataFrame

    wf.GetColumnFromDataFrame_2 = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name=GetColumnFromDataFrame_2__column_name
    )
    from pyiron_nodes.dataframe import GetColumnFromDataFrame

    wf.GetColumnFromDataFrame_3 = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name=GetColumnFromDataFrame_3__column_name
    )
    from pyiron_nodes.dataframe import GetColumnFromDataFrame

    wf.GetColumnFromDataFrame_1 = GetColumnFromDataFrame(
        df=wf.GetRowsFromDataFrame, column_name=GetColumnFromDataFrame_1__column_name
    )
    from pyiron_nodes.dataframe import ApplyFunctionToSeriesNew

    wf.ApplyFunctionToSeries_2 = ApplyFunctionToSeriesNew(
        series=wf.GetColumnFromDataFrame_2,
        function=wf.LinearInterpolationDescriptor,
        store=ApplyFunctionToSeries_2__store,
    )
    from pyiron_nodes.math import Subtract

    wf.Subtract = Subtract(x=wf.GetColumnFromDataFrame_1, y=wf.GetColumnFromDataFrame_3)
    from pyiron_nodes.math import PseudoInverse

    wf.PseudoInverse = PseudoInverse(matrix=wf.ApplyFunctionToSeries_2)
    from pyiron_nodes.math import Sum

    wf.Sum = Sum(x=wf.ApplyFunctionToSeries_2, axis=Sum__axis)
    from pyiron_nodes.math import DotProduct

    wf.DotProduct = DotProduct(
        a=wf.PseudoInverse, b=wf.Subtract, store=DotProduct__store
    )


    return wf.DotProduct

In [ ]:
assyst_linear_fit3().pull()

In [ ]:
xx

import calphy

In [ ]:
import pyiron_nodes as pn
import pandas as pd
import numpy as np
from pyiron_nodes.dataframe import ApplyFunctionToSeriesNew
from pyiron_workflow import Workflow

wf = Workflow("test")

structure = pn.atomistic.structure.build.CubicBulkCell('Al').run()

series = pd.Series([structure, structure]) 


wf.node = pn.atomistic.structure.calc.LinearInterpolationDescriptor()

wf.Apply_Function = ApplyFunctionToSeriesNew(
    series=series,
    function=wf.node,
    store=False,
)

wf.Apply_Function.pull()
# wf.node.run()

In [ ]:
list(node.kwargs.keys())[0]

In [ ]:
import pandas as pd
import numpy as np

pd.Series([1,2]).apply(np.sin)

In [ ]:
xx

In [ ]:
from structuretoolkit import get_neighbors, 

get_neighbors

In [ ]:
import numpy as np

x_min = 0
x_max = 1
steps = 11
x, dx = np.linspace(x_min, x_max, steps, retstep=True)
y = np.zeros(steps)

x0 = 0.3001
i_left = int((x0 - x_min) * steps) 

w = (x0 - x[i_left])/dx
print (i_left, x, w)
y[i_left] += 1-w
y[i_left + 1] = w
x[i_left], x[i_left+1]
np.sum(x * y)

In [ ]:
import pyiron_nodes as pn

pn.atomistic.structure.build.CubicBulkCell('Al').run()

In [ ]:


import pyiron_nodes as pn

import numpy as np
from scipy.stats import norm

def gaussian_weighted_histogram(data, bins=50, sigma=1.0):
    """
    Compute a Gaussian-weighted histogram for a list of floats.

    Parameters:
        data (list or np.ndarray): Input data (list of floats).
        bins (int): Number of bins for the histogram.
        sigma (float): Standard deviation of the Gaussian kernel.

    Returns:
        bin_centers (np.ndarray): Centers of the histogram bins.
        weighted_histogram (np.ndarray): Gaussian-weighted histogram values.
    """
    # Create histogram bins
    bin_edges = np.linspace(min(data), max(data), bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # Initialize the weighted histogram
    weighted_histogram = np.zeros_like(bin_centers)

    # Apply Gaussian weighting for each data point
    for x in data:
        weights = norm.pdf(bin_centers, loc=x, scale=sigma)
        weighted_histogram += weights

    return bin_centers, weighted_histogram


bulk = pn.atomistic.structure.build.Bulk('Al', cubic=True)

bulk.kwargs

In [ ]:
pf = gui.PyironFlow(['assyst',  'linearfit2', 'landau2', 'svd_solver', 'linear_fit',  'plot_sin', 'supercell_conv']) 
pf.gui

In [ ]:
xx

ToDo (key priorities):
- renaming of labels
- grouping with renaming of group names
- sorting and search in node tree
- js gui input - sign fails (e.g. -1) gives json reading error (solved in gui.py)
- run only needed nodes (where no hashed output is available)


In [ ]:
None

In [ ]:
import json

my_str = "{\"label\":\"Identity\",\"handle\":0,\"value\":\"-1\"}"

json.loads(my_str)

In [ ]:
import numpy as np

lst = []
for i in [[0], [1,1]]:
    lst += list(np.array(i))
lst

np.sqrt(0.000496688)

In [ ]:
import numpy as np

np.append()


In [ ]:
from pyiron_workflow import Workflow, as_macro_node
import pyiron_nodes as pn

pn.atomistic.ml_potentials.fitting.linear_ace

In [ ]:
@as_macro_node("phase_data")
def ComputPhaseDiagram(filename: str="MgCaFreeEnergies.pckl.gz", T_min:int=300, T_max:int=1100, T_steps=20):
    wf = Workflow("PhaseDiagram")
    wf.read_data = pn.utilities.ReadDataFrame(filename=filename, compression="gzip")
    wf.phases_from_df = pn.atomistic.thermodynamics.landau.phases.PhasesFromDataFrame(dataframe=wf.read_data)
    wf.temperatures = pn.math.Linspace(x_min=T_min, x_max=T_max, num_points=T_steps, endpoint=True)
    wf.calc_phase_diagram = pn.atomistic.thermodynamics.landau.plot.CalcPhaseDiagram(phases=wf.phases_from_df.outputs.phase_list, temperatures=wf.temperatures, refine=True)
    return wf.calc_phase_diagram

In [ ]:
ComputPhaseDiagram().run()

In [ ]:
from pyace import PyACECalculator
import pyiron_nodes
from pyiron_workflow import Workflow

In [ ]:
wf = Workflow('linearfit2')

wf.ParameterizePotentialConfig = pyiron_nodes.atomistic.ml_potentials.fitting.linearfit.ParameterizePotentialConfig(number_of_functions_per_element=100) 
wf.ReadPickledDatasetAsDataframe = pyiron_nodes.atomistic.ml_potentials.fitting.linearfit.ReadPickledDatasetAsDataframe(file_path="mgca.pckl.tgz") 
wf.SplitTrainingAndTesting = pyiron_nodes.atomistic.ml_potentials.fitting.linearfit.SplitTrainingAndTesting(data_df=wf.ReadPickledDatasetAsDataframe) 
wf.RunLinearFit = pyiron_nodes.atomistic.ml_potentials.fitting.linearfit.RunLinearFit(df_test=wf.SplitTrainingAndTesting.outputs.df_testing, 
                                                                                      df_train=wf.SplitTrainingAndTesting.outputs.df_training, potential_config=wf.ParameterizePotentialConfig) 

In [ ]:
fit = wf.RunLinearFit.pull()

In [ ]:
ace = PyACECalculator(fit)
ace.basis.basis_coeffs;

In [ ]:
from pyiron_nodes.atomistic.structure.build import Bulk

structure = Bulk("Ca", cubic=True).run()

In [ ]:
ace.calculate(atoms=structure)

In [ ]:
df_train = wf.SplitTrainingAndTesting.outputs.df_testing.value
potential_config = wf.ParameterizePotentialConfig.run()

In [ ]:
from pyace.linearacefit import LinearACEFit, LinearACEDataset
from pyace import create_multispecies_basis_config

from pyiron_snippets.logger import logger

logger.setLevel(30)
verbose = False

elements_set = set()
for at in df_train["ase_atoms"]:
    elements_set.update(at.get_chemical_symbols())

elements = sorted(elements_set)
potential_config.elements = elements
potential_config_dict = potential_config.to_dict()

bconf = create_multispecies_basis_config(potential_config_dict)

train_ds = LinearACEDataset(bconf, df_train)
train_ds.construct_design_matrix(verbose=verbose)

In [ ]:
mat = train_ds.design_matrix

In [ ]:
import numpy as np
import matplotlib.pylab

# np.linalg.svd(mat)

In [ ]:
%%time
u, s, vh = np.linalg.svd(mat, full_matrices=False)

In [ ]:
plt.plot(vh[96]);

In [ ]:
svd_mat = np.zeros(mat.shape)
norm_list = []
for i in range(len(s)):
    svd_mat += np.outer(u.T[i], vh[i]) * s[i]
    norm_list.append(np.linalg.norm(svd_mat - mat))

In [ ]:
plt.plot(norm_list, label='norm')
plt.plot(s, label='s')
plt.yscale('log')
plt.xscale('log')
plt.legend()

In [ ]:
df_train.energy_corrected

In [ ]:
# Reference data
training_number_of_atoms = df_train.NUMBER_OF_ATOMS.to_numpy()
training_energies = df_train.energy_corrected.to_numpy()

np.sum(training_number_of_atoms)

In [ ]:
mat.shape

In [ ]:
train_ds.construct_target_vector()

In [ ]:
len(train_ds.get_energies_per_atom())

In [ ]:
train_ds.get_energies_per_atom??

In [ ]:
train_ds.get_target_vector??